In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb.tensor as qtn
import quimb as qu
import cotengra as ctg
import autoray as ar
from time_compressed_algo import register_ as reg
from time_compressed_algo import algo_cooling as algo
from time_compressed_algo import quf
import time
import numpy as np
from quimb.tensor.belief_propagation.l2bp import L2BP
from tqdm import tqdm
import gate_arb
from gate_arb import TensorNetworkGenVector

In [2]:
reg.reg_complex_svd()
import torch
to_backend = algo.backend_torch(device = "cpu", dtype = torch.complex128, requires_grad=False)
to_backend_c = algo.backend_torch(device = "cpu", dtype = torch.complex128, requires_grad=False)

opt = algo.opt_(progbar=False)

In [3]:
#ITF params
J = 1
h = 3.05
delta = 0.05
Lx, Ly = 4, 4
L = Lx * Ly
edges = qtn.edges_2d_square(Lx=Lx, Ly=Ly, cyclic=False)


In [4]:
map_1d_2d = { Lx*j+i: (i,j)   for i in range(Lx) for j in range(Ly)}
map_2d_1d = { (i,j):Lx*j+i   for i in range(Lx) for j in range(Ly)}
map_tags = { f"I{i},{j}":f"I{Lx*j+i}"  for i in range(Lx) for j in range(Ly)}
map_inds = { f"k{i},{j}":f"k{Lx*j+i}"  for i in range(Lx) for j in range(Ly)}
site_tags = [ f"I{i}" for i in range(L)]
sites = sorted({ (map_2d_1d[site],) for edge in edges for site in edge})
N = len(sites)
print(N)
edges_ = []
for edge in edges:
    x, y = edge
    edges_.append( (map_2d_1d[x], map_2d_1d[y])  )

16


In [5]:
# mpo = quf.mpo_ITF_2d(Lx, Ly, data_type="float64", chi=200, cutoff_val=1.0e-12, field=h, sign="-", print_=False)

In [6]:
Harr_params = [ quf.random_haar_qubit(seed=1)   for i in range(L)]
peps = quf.peps_haar(Lx=Lx, Ly=Ly, dtype="complex128", Harr_params = Harr_params)
peps.apply_to_arrays(to_backend)
peps.retag_(map_tags)
peps.reindex_(map_inds)

peps.view_as_(TensorNetworkGenVector,  site_tag_id='I{}',site_ind_id='k{}',)


TensorNetworkGenVector(tensors=16, indices=40)

In [7]:
# norm_ = (peps.H & peps).contract(all, optimize=opt)
# norm_

In [8]:
%%time
chi = 4
gauges = {}

depth_total = 20
pbar = tqdm(total=depth_total, desc="SU", ncols=100, dynamic_ncols=True)
est_norm = 1.
exponent = 0
renorm =  True
Infidelity_l = []
for depth in range(depth_total):
    peps_raw = peps.copy()
    if gauges:
        gauges_ =  { u:v*1.  for u, v in gauges.items()}
    else:
        gauges_ = {}

    
    if depth == 0 or depth==depth_total:
        rx = qtn.circuit.rx_gate_param_gen([-h*delta])
    else:
        rx = qtn.circuit.rx_gate_param_gen([-2*h*delta])
    
    rzz = qtn.circuit.rzz_param_gen([-2*J*delta])

    for count, site in enumerate(sites):
        peps.gate_simple_(to_backend(rx),site, gauges=gauges)
        peps_raw.gate_simple_(to_backend(rx),site, gauges=gauges_)

    
    for where in edges_:
        peps.gate_simple_(to_backend(rzz),where,gauges=gauges,max_bond=chi, cutoff=1.e-12,
                          cutoff_mode='rsum2',
            renorm=renorm,
        )
        peps_raw.gate_simple_(to_backend(rzz),where, gauges=gauges_, max_bond=2*chi, cutoff=1.e-12, 
                              cutoff_mode='abs', #'rsum2',
            renorm=renorm,
        )
    
    for count, site in enumerate(sites):
        peps.gate_simple_(to_backend(rx),site, gauges=gauges, )
        peps_raw.gate_simple_(to_backend(rx), site, gauges=gauges_)

    peps_ = peps.copy()
    print(peps.exponent)
    peps_.gauge_simple_insert(gauges)
    print(peps_.exponent)
    peps_raw.gauge_simple_insert(gauges_)

    norm = quf.peps_norm(peps_, opt=opt, copt=None, chi=8, mode="exact",
            max_separation=1, 
            sequence = ['xmin', 'xmax', 'ymin', 'ymax'],
            progbar=False,
            )
    norm_ = quf.peps_norm(peps_raw, opt=opt, copt=None, chi=8, mode="exact",
            max_separation=1, 
            sequence = ['xmin', 'xmax', 'ymin', 'ymax'],
            progbar=False,
            )

    print("norm", complex(norm).real, complex(norm_).real)

    f_ = quf.loss_peps_g(peps_, peps_raw, opt, None, cost_f="fid", chi_bmps=4, mode = "exact", progbar=False)
    f_ = complex(f_).real
    Infidelity_l.append(1 - complex(f_).real)
    bp = L2BP(peps_, optimize=opt, site_tags=site_tags)
    bp.run(max_iterations=420, tol=1.e-7, progbar=False, diis=True)
    est_norm = complex(bp.contract()).real
    print("bp_norm", complex(est_norm * 10**(2*peps.exponent)).real, "f", complex(f_).real)
    print("inf", np.prod(Infidelity_l), peps_.exponent)
    
    pbar.set_postfix({
            "bpnorm": f"{est_norm:.4e}", "norm_": f"{norm_:.4e}", "norm": f"{norm:.4e}",
        "f": f"{f_:.4e}", "inf": f"{ np.prod(Infidelity_l):.4e}"
        })
    pbar.update(1)
pbar.close()



SU:   0%|                                                                                                       | 0/20 [00:00<?, ?it/s]

tensor(1.4470e-11, dtype=torch.float64)
tensor(1.4470e-11, dtype=torch.float64)
norm 0.9999999999999892 0.9999999999999892


SU:   5%| | 1/20 [00:00<00:03,  5.47it/s, bpnorm=1.0000e+00, norm_=1.0000e+00-3.1955e-17j, norm=1.0000e+00-3.1955e-17j, f=-4.4409e-16, 

bp_norm 0.999999999999987 f -4.440892098500626e-16
inf 1.0000000000000004 tensor(1.4470e-11, dtype=torch.float64)
tensor(1.5528e-11, dtype=torch.float64)
tensor(1.5528e-11, dtype=torch.float64)
norm 1.0000000000024312 1.0000000000024312


SU:  10%| | 2/20 [00:00<00:03,  4.56it/s, bpnorm=1.0000e+00, norm_=1.0000e+00-1.4371e-16j, norm=1.0000e+00-1.4371e-16j, f=-4.4409e-16, 

...

bp_norm 0.9969095422015377 f 0.0015147767617643693
inf 0.9908280403377843 tensor(-0.0007, dtype=torch.float64)
tensor(-0.0009, dtype=torch.float64)
tensor(-0.0009, dtype=torch.float64)
norm 0.9433251825683945 0.9249419802386688


SU:  95%|▉| 19/20 [00:08<00:00,  1.83it/s, bpnorm=1.0000e+00, norm_=9.2494e-01-3.0104e-16j, norm=9.4333e-01+7.6541e-17j, f=1.3972e-03, 

bp_norm 0.9960779963012057 f 0.0013971643085256558
inf 0.9894436907639379 tensor(-0.0009, dtype=torch.float64)
tensor(-0.0010, dtype=torch.float64)
tensor(-0.0010, dtype=torch.float64)
norm 0.9429841201258923 0.9424927461392185


SU: 100%|█| 20/20 [00:09<00:00,  2.09it/s, bpnorm=1.0000e+00, norm_=9.4249e-01+3.7950e-17j, norm=9.4298e-01+1.7329e-16j, f=1.9074e-03, 

bp_norm 0.995199523376397 f 0.0019074205447678905
inf 0.9875564055402838 tensor(-0.0010, dtype=torch.float64)
CPU times: user 2min 2s, sys: 2.66 s, total: 2min 4s
Wall time: 9.56 s


In [9]:
peps.exponent, 10**(2*peps.exponent)

(tensor(-0.0010, dtype=torch.float64), tensor(0.9952, dtype=torch.float64))

In [10]:
peps.gauge_simple_insert(gauges)

bp = L2BP(peps, optimize=opt, site_tags=site_tags)
bp.run(max_iterations=420, tol=1.e-7, progbar=False, diis=True)
est_norm = complex(bp.contract()).real
est_norm

1.0000004741649589

In [11]:
(peps.H | peps).contract(all, optimize=opt)

tensor(0.9430+1.7329e-16j, dtype=torch.complex128)